In [29]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim

# New stuff.
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence
DO_VAE=True

In [27]:
# setup
train_dataset = datasets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = datasets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())
torch.manual_seed(3435)
if DO_VAE:
    train_img = torch.stack([torch.bernoulli(d[0]) for d in train_dataset])
    test_img = torch.stack([torch.bernoulli(d[0]) for d in test_dataset])
else:
    train_img = torch.stack([d[0] for d in train_dataset])
    test_img = torch.stack([d[1] for d in test_dataset])
train_img = torch.squeeze(train_img)
test_img = torch.squeeze(test_img)
train_label = torch.LongTensor([d[1] for d in train_dataset])
test_label = torch.LongTensor([d[1] for d in test_dataset])

val_img = train_img[-10000:].clone()
val_label = train_label[-10000:].clone()
train_img = train_img[:-10000]
train_label = train_label[:-10000]

train = torch.utils.data.TensorDataset(train_img, train_label)
val = torch.utils.data.TensorDataset(val_img, val_label)
test = torch.utils.data.TensorDataset(test_img, test_label)
BATCH_SIZE = 100
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

In [25]:
encoder_mlp = MLPEncoder(hidden_dim=200, latent_dim=10)
decoder_mlp = MLPDecoder(hidden_dim=200, latent_dim=10)
vae = NormalVAE(encoder_mlp, decoder_mlp)
models = [encoder_mlp, decoder_mlp, vae]
lm_evaluator = LatentModelEvaluator(models)
lm_trainer = LatentModelTrainer(models, lrn_rate=0.05, optimizer = optim.SGD)
lm_trainer.train(train_loader, le=lm_evaluator, val_loader=val_loader,
                nnum_epochs=10)

torch.Size([100, 28, 28]) torch.Size([100])


In [18]:
print(train_label.size())
for d in train_dataset:
    print(d[0].size(), d[1])
    break

torch.Size([50000])
torch.Size([1, 28, 28]) 
 5
[torch.LongTensor of size ()]



In [26]:
train_loader.init_epoch()

AttributeError: 'DataLoader' object has no attribute 'init_epoch'

In [28]:
len(train_loader)

500